In [1]:
import os, sys
import glob
import time
from datetime import timedelta, datetime
import json
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
#setting offilne
plotly.offline.init_notebook_mode(connected=True)
import pyarrow.parquet as pq
from gezi import tqdm
import gezi

# corpus of vid

In [2]:
with open(f'../input/all/corpus.txt', 'w') as out_all:
  with open(f'../input/all/corpus_novalid.txt', 'w') as out_novalid:
    with open(f'../input/all/corpus_train.txt', 'w') as out_train:
      for k in tqdm(range(30)):
        d = gezi.read_parquet(f'../input/train/part_{k + 1}/user.parquet')
        d.watch = d.watch.apply(json.loads)
        watches = d.watch.values
        with open(f'../input/train/part_{k + 1}/corpus.txt', 'w') as out:
          for i in range(len(watches)):
            l = []
            for j in range(len(watches[i])):
              l.append(str(watches[i][j][1]))
            if l:
              print(' '.join(l), file=out)
              print(' '.join(l), file=out_all)
              if i != 30:
                print(' '.join(l), file=out_novalid)
                print(' '.join(l), file=out_train)
      d = gezi.read_parquet('../input/eval/user.parquet')
      d.watch = d.watch.apply(json.loads)
      watches = d.watch.values
      with open(f'../input/eval/corpus.txt', 'w') as out:
        for i in tqdm(range(len(watches))):
          l = []
          for j in range(len(watches[i])):
            l.append(str(watches[i][j][1]))
          if l:
            print(' '.join(l), file=out)
            print(' '.join(l), file=out_novalid)
            print(' '.join(l), file=out_all)

100%|██████████| 156273/156273 [00:08<00:00, 19481.63it/s]


# corpus of vid without click data

In [2]:
clicks = pd.read_csv('../input/all/clicks.csv')
clicks = set(zip(clicks.did.values, clicks.vid.values))
with open(f'../input/all/corpus_filter.txt', 'w') as out:
  with open(f'../input/all/corpus_filter_train.txt', 'w') as out_train:
    for k in tqdm(range(30)):
      d = gezi.read_parquet(f'../input/train/part_{k + 1}/user.parquet')
      dids = d.did.values
      d.watch = d.watch.apply(json.loads)
      watches = d.watch.values
      for i in range(len(watches)):
        did = dids[i]
        l = []
        for j in range(len(watches[i])):
          vid = watches[i][j][1]
          if not (did, vid) in clicks:
            l.append(str(vid))
        if l:
          print(' '.join(l), file=out)
        if i != 30:
          print(' '.join(l), file=out_train)
    d = gezi.read_parquet('../input/eval/user.parquet')
    d.watch = d.watch.apply(json.loads)
    watches = d.watch.values
    for i in tqdm(range(len(watches))):
      l = []
      for j in range(len(watches[i])):
        vid = watches[i][j][1]
        if not (did, vid) in clicks:
          l.append(str(vid))
      if l:
        print(' '.join(l), file=out)

100%|██████████| 156273/156273 [00:08<00:00, 17576.82it/s]


# corpus add click data

In [6]:
dcs = []
for k in tqdm(range(30)):
  d = gezi.read_parquet(f'../input/train/part_{k + 1}/context.parquet')
  d = d[d.label==1][['did', 'vid', 'timestamp']]
  d['day'] = k + 1
  dcs += [d]
dc = pd.concat(dcs)

100%|██████████| 30/30 [00:33<00:00,  1.09s/it]


In [12]:
dc = dc.groupby('did').agg(vids=('vid', lambda x: list(x))).reset_index()

In [13]:
dc

,did,vids
0,000018568561880a0ab2e712c24fd935,"[1933869142, 622642288, 3913676406, 2872596680]"
1,000022506218ddedd0d064a96d07b6c5,"[1653383727, 1867735501, 2315871907]"
2,00002ad59519acc2b153c62da76a0fd0,[3592760380]
3,00002f7f7abc1e5aae45f774a3a44478,"[1790097910, 3293210169, 3521169412, 686940390..."
4,00003fc63ff11ea88d241d52d9bcc83a,[1907952065]
...,...,...
1177345,ffffd0ff4ed3b61458e1473373d9ce02,[622642288]
1177346,ffffd77a99fc2d36f60faf962c3225ee,[2436335535]
1177347,ffffeaa8cc4da28613264765faf6edd1,"[2314749750, 609745040]"
1177348,ffffed8bb4589077e9f33b365a1a5a4a,[2449410198]


In [15]:
m = {}
for _, row in tqdm(dc.iterrows(), total=len(dc)):
  m[row['did']] = row['vids']

100%|██████████| 1177350/1177350 [04:10<00:00, 4701.70it/s]


In [17]:
clicks = pd.read_csv('../input/all/clicks.csv')
clicks = set(zip(clicks.did.values, clicks.vid.values))
with open(f'../input/all/corpus_add.txt', 'w') as out:
  with open(f'../input/all/corpus_add_train.txt', 'w') as out_train:
    for k in tqdm(range(30)):
      d = gezi.read_parquet(f'../input/train/part_{k + 1}/user.parquet')
      dids = d.did.values
      d.watch = d.watch.apply(json.loads)
      watches = d.watch.values
      for i in range(len(watches)):
        did = dids[i]
        clicks = m[did] if did in m else []
        clicks = [str(x) for x in clicks]
        l = []
        for j in range(len(watches[i])):
          vid = watches[i][j][1]
          if not (did, vid) in clicks:
            l.append(str(vid))
        l += clicks
        if l:
          print(' '.join(l), file=out)
        if i != 30:
          print(' '.join(l), file=out_train)
    d = gezi.read_parquet('../input/eval/user.parquet')
    d.watch = d.watch.apply(json.loads)
    watches = d.watch.values
    for i in tqdm(range(len(watches))):
      l = []
      for j in range(len(watches[i])):
        vid = watches[i][j][1]
        if not (did, vid) in clicks:
          l.append(str(vid))
      if l:
        print(' '.join(l), file=out)

100%|██████████| 156273/156273 [00:05<00:00, 27528.68it/s]


# corpus of title from video info

In [8]:
vinfo = pq.read_table('../input/train/raw.parquet').to_pandas()

In [9]:
vinfo

,vid,image_emb,title,story
0,5893106,"-8.26598072052,1.63619112968,17.9674415588,6.9...","199575,199576,40564",
1,10423101,"-3.63512468338,2.38069868088,14.9096345901,1.2...","717361,318",
2,15819492,"0.107854895294,3.20008993149,20.3623485565,-0....","396,1705,31495,3424,77035,85420,28,17970,9300,...",
3,16814925,"-11.0617599487,1.99434304237,14.9591712952,1.2...","8579,18147,73,227",
4,20360674,"6.6802148819,1.83605599403,15.9008598328,3.244...","22485,118989,12418,396611,61891,3029,26353,396...",
...,...,...,...,...
183788,4276491220,"-4.29484510422,6.66334295273,16.4290809631,-4....","3072,34976,173488,1864,6276,492635,58,19175,24...",
183789,4283547666,"-4.81997919083,13.6302375793,14.3650541306,6.9...","266947,7450,58397,42,58167,44,2577,5",
183790,4283640696,"-0.665687978268,11.5875854492,9.86795520782,3....","406009,91149,52611,66378,290,355096",
183791,4290887658,"2.51159644127,2.58366537094,9.91071224213,4.36...","42,128095,44,188981,28,6705,87381,2679,59,3316...",


In [10]:
with open(f'../input/all/corpus_words.txt', 'w') as out:
  titles = vinfo.title.values
  stories = vinfo.story.values
  for i in tqdm(range(len(vinfo))):
    if ','  in titles[i]:
      print(titles[i].replace(',', ' '), file=out)
    if ','  in stories[i]:
      print(stories[i].replace(',', ' '), file=out)

100%|██████████| 183793/183793 [00:00<00:00, 370717.69it/s]


In [11]:
vinfo['title_words'] = vinfo.title.apply(lambda x: len(x.split(',')))

In [12]:
vinfo.title_words.describe()

count    183793.000000
mean         10.977856
std           4.995826
min           1.000000
25%           8.000000
50%          11.000000
75%          14.000000
max          40.000000
Name: title_words, dtype: float64

In [13]:
len(vinfo[vinfo.title_words>10]) / len(vinfo)

0.5644828693149359

# corpus of stars

In [15]:
with open('../input/all/corpus_stars.txt', 'w') as out:
  for i in tqdm(range(30)):
    d = gezi.read_parquet(f'../input/train/part_{i + 1}/item.parquet')
    for stars in d.stars:
      content = ' '.join(map(str, stars))
      if len(content.strip().split()) > 1:
        print(content, file=out)
  d = gezi.read_parquet(f'../input/eval/item.parquet')
  for stars in d.stars:
    content = ' '.join(map(str, stars))
    if len(content.strip().split()) > 1:
      print(content, file=out)

100%|██████████| 30/30 [00:19<00:00,  1.74it/s]


In [16]:
d

,vid,title_length,timestamp,stars,cid,class_id,is_intact,second_class,duration,ctr,vv
0,5893106,25,1411552000,[],600008821,3174145329,2234347078,0,86.110001,0.000,1004.0
1,15819492,98,1407204002,"[2950753864, 3600550670, 3606342894, 333836411...",638675564,1413776458,2234347078,0,6954.200195,0.065,1392.0
2,16814925,21,1380506136,[],511097257,3174145329,2234347078,0,317.190002,0.000,4360.0
3,38297769,112,1425690005,"[3115812425, 2144664037, 1221620183, 325915182...",2168442617,1413776458,472867291,0,42.509998,0.000,5832.0
4,53280242,85,1367558225,"[591536023, 1613895128, 2189138809, 559740265,...",155617312,3443952599,3172394293,0,376.049988,0.000,248.0
...,...,...,...,...,...,...,...,...,...,...,...
56531,4204251528,70,1360118075,"[1074701228, 4106478470]",2239251993,1413776458,2234347078,0,771.719971,0.000,56.0
56532,4255628613,33,1413545288,[1134598214],1016220003,1820266297,2234347078,0,35.970001,0.000,480.0
56533,4261704198,32,1382493214,[],1611517696,3174145329,2234347078,0,115.540001,0.000,36.0
56534,4272496705,80,1387029762,"[3309131045, 2561243119, 184102567, 2880021111...",435329116,1820266297,472867291,0,115.540001,0.000,92.0
